# Suggested solution to Exercise 11, Week 7

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.13.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


In [2]:
K, (u1,u2) = RationalFunctionField(QQ,["u1","u2"])

# Alternatively
#U, (u1,u2) = polynomial_ring(QQ,["u1","u2"])
#K = fraction_field(U)

(Rational function field over QQ, AbstractAlgebra.Generic.RationalFunctionFieldElem{QQFieldElem, QQMPolyRingElem}[u1, u2])

In [3]:
R, (x1,x2,lambda) = polynomial_ring(K,["x1","x2","lambda"])

(Multivariate polynomial ring in 3 variables over rational function field, AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.RationalFunctionFieldElem{QQFieldElem, QQMPolyRingElem}}[x1, x2, lambda])

In [4]:
F = [x1^2*x2^2 - 3*x1^2 - 3*x2^2 + 5, 2*x1*x2^2 - 6*x1 - lambda*(x1-u1), 2*x1^2*x2 - 6*x2 - lambda*(x2-u2)]

3-element Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.RationalFunctionFieldElem{QQFieldElem, QQMPolyRingElem}}}:
 x1^2*x2^2 - 3*x1^2 - 3*x2^2 + 5
 2*x1*x2^2 - x1*lambda - 6*x1 + u1*lambda
 2*x1^2*x2 - x2*lambda - 6*x2 + u2*lambda

## Part (a): Upper bounds

The Bézout bound is $4\cdot 3\cdot 3=36$.

The BKK bound in $\mathbb{C}^3$ is 12. 

To compute this, we artifically make sure that each polynomial has a constant term (this corresponds to adding the origin as a vertex to the Newton polytopes). In this case we can do this by adding `[0,1,1]` to `F`.

In [5]:
import Combinatorics: powerset
function my_mixed_volume(list_of_polytopes)
    n = length(list_of_polytopes)
    return sum([(-1)^(n-length(I))*volume(sum(list_of_polytopes[I])) 
        for I in powerset(collect(1:n)) if !isempty(I)])
end;

In [6]:
# Add one to the second and third polynomial
F + [0,1,1]

3-element Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.RationalFunctionFieldElem{QQFieldElem, QQMPolyRingElem}}}:
 x1^2*x2^2 - 3*x1^2 - 3*x2^2 + 5
 2*x1*x2^2 - x1*lambda - 6*x1 + u1*lambda + 1
 2*x1^2*x2 - x2*lambda - 6*x2 + u2*lambda + 1

In [7]:
# Compute the Newton polytope of each of the polynomials in the system
newton_polytope.(F+[0,1,1])

3-element Vector{Polyhedron{QQFieldElem}}:
 Polyhedron in ambient dimension 3
 Polyhedron in ambient dimension 3
 Polyhedron in ambient dimension 3

In [8]:
# Compute the mixed volume
my_mixed_volume(newton_polytope.(F+[0,1,1]))

12

## Part (b): Computation of the EDD degree

To actually compute the generic number of roots we can follow the following strategy:

**Step 1:** Compute a parametric Gröbner basis of $I:=\langle f, \partial_x f, \partial_y f\rangle$ over $\mathbb{Q}(u_1,u_2)$, and count the number of standard monomials. 

For generic $u\in\mathbb{C}^2$, this Gröbner basis will specialize to a Gröbner basis for $I_u=\langle f_u, \partial_x f_u, \partial_y f_u\rangle\subseteq\mathbb{C}[x,y,\lambda]$, and therefore give the dimension of the specialized coordinate ring $\mathbb{C}[x,y,\lambda]/I_u$.

**Step 2:** Check that $\langle f, \partial_x f, \partial_y f\rangle$ is radical over $\mathbb{Q}(u_1,u_2)$. 

This will imply that for generic generic $u\in\mathbb{C}^2$, the specialized ideal $\langle f_u, \partial_x f_u, \partial_y f_u\rangle$ is radical. (Proving this is a bit beyond the scope of the course, but the idea is to use Seidelberg's lemma - google this if you're up for a challenge!)

**Conclusion:** In this specific case, we find that the ideal is radical over $\mathbb{Q}(u_1,u_2)$, and that we have 12 standard monomials:
$$1,\quad x_1,\quad x_1^2,\quad x_2,\quad x_1x_2,\quad x_2^2,\quad \lambda,\quad x_1\lambda,\quad x_2\lambda,\quad \lambda^2,\quad x_2\lambda^2,\quad \lambda^3$$
Hence, the ED degree is 12.

In [9]:
I = ideal(F)
G = groebner_basis(I, ordering=degrevlex(R))

Gröbner basis with elements
1 -> (u1*u2^2 - 3*u1)*x1*lambda^2 + (u1^2*u2 - 3*u2)*x2*lambda^2 + (36*u2^2 - 60)*x1^2 + 16*u1*u2*x1*x2 + (36*u1^2 - 60)*x2^2 + (6*u1^3 - 10*u1)*x1*lambda + (6*u2^3 - 10*u2)*x2*lambda + (-u1^2*u2^2 + 5)*lambda^2 + 48*u1*x1 + 48*u2*x2 + (-18*u1^2 - 18*u2^2 + 60)*lambda - 60*u1^2 - 60*u2^2 + 280
2 -> x2^2*lambda - 6*x1^2 - u2*x2*lambda + 10
3 -> 16*u1*u2*x1*x2*lambda + (24*u1^2*u2^2 - 120)*x2^2*lambda + (2*u1^3*u2^2 + 6*u1^3 + 2*u1*u2^4 - 18*u1*u2^2 + 8*u1)*x1*lambda^2 + (2*u1^4*u2 + 2*u1^2*u2^3 - 18*u1^2*u2 + 6*u2^3 + 8*u2)*x2*lambda^2 + (u1^2*u2^2 - 3*u1^2 - 3*u2^2 + 5)*lambda^3 + (-72*u1^2*u2^2 + 216*u1^2 - 216*u2^2 + 360)*x1^2 + (32*u1^3*u2 + 32*u1*u2^3 - 192*u1*u2)*x1*x2 + (72*u1^2*u2^2 - 216*u1^2 + 216*u2^2 - 360)*x2^2 + (12*u1^3*u2^2 - 36*u1^3 + 52*u1*u2^2 - 60*u1)*x1*lambda + (-12*u1^2*u2^3 + 52*u1^2*u2 - 36*u2^3 + 60*u2)*x2*lambda + (-2*u1^4*u2^2 - 2*u1^2*u2^4 + 12*u1^2*u2^2 - 18*u1^2 - 18*u2^2 + 60)*lambda^2 + (96*u1*u2^2 - 288*u1)*x1 + (96*u1^2*u2 -

In [10]:
leading_monomial.(G)

10-element Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.RationalFunctionFieldElem{QQFieldElem, QQMPolyRingElem}}}:
 x1*lambda^2
 x2^2*lambda
 x1*x2*lambda
 x1^2*lambda
 x2^3
 x1*x2^2
 x1^2*x2
 x1^3
 lambda^4
 x2*lambda^3

In [11]:
is_radical(I)

true

<div class="alert alert-block alert-info">
<strong>Remark:</strong> The fact that the ED degree and the mixed volume were the same in this exercise was not a coincidence! In a certain sense, this happens for "almost all" curves, which follows by the main result in the paper <a href="https://arxiv.org/abs/2012.06350">Euclidean distance degree and mixed volume</a> by Paul Breiding, Frank Sottile, and James Woodcock. You can also watch this very nice <a href="https://youtu.be/6ZXN7OdUe9k?si=fE37CF-xpn4TXgaB">talk</a> on the same topic by Paul Breiding.
</div>
